# Generating Shakespearean Text with Character Based RNNs

Problem Statement: Given a character or sequence of characters, we want to predict the next character at each time step. Model is trained to follow a language similar to the works of Shakespeare. The tinyshakespear dataset is used for training.

In [23]:
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
import os
import time

In [24]:
#check if decoding is needed: text may need to be decoded as utf-8
text = open('/kaggle/input/complete-works-of-rabindranath-tagore/txt/poem.txt', 'r').read() 
print(text[:200])

বজাও রে মোহন বাঁশি।
সারা দিবসক
বিরহদহনদুখ,
মরমক তিয়াষ নাশি।
রিঝমনভেদন
বাঁশরিবাদন
কঁহা শিখলি রে কান?
হানে থিরথির
মরমঅবশকর
লহু লহু মধুময় বাণ।
ধসধস করতহ
উরহ বিয়াকুলু,
ঢুলু ঢুলু অবশনয়ান ;
কত কত বরষক
বাত স


In [25]:
#Find Vocabulary (set of characters)
vocabulary = sorted(set(text))
print('No. of unique characters: {}'.format(len(vocabulary)))

No. of unique characters: 139


## Preprocessing Text

In [26]:
#character to index mapping
char2index = {c:i for i,c in enumerate(vocabulary)}
int_text = np.array([char2index[i] for i in text])

#Index to character mapping
index2char = np.array(vocabulary)

In [27]:
#Testing
print("Character to Index: \n")
for char,_ in zip(char2index, range(65)):
    print('  {:4s}: {:3d}'.format(repr(char), char2index[char]))

print("\nInput text to Integer: \n")
print('{} mapped to {}'.format(repr(text[:20]),int_text[:20])) #use repr() for debugging

Character to Index: 

  '\n':   0
  ' ' :   1
  '!' :   2
  '"' :   3
  "'" :   4
  '(' :   5
  ')' :   6
  ',' :   7
  '-' :   8
  '.' :   9
  '1' :  10
  '2' :  11
  '6' :  12
  '7' :  13
  '9' :  14
  ':' :  15
  ';' :  16
  '?' :  17
  'B' :  18
  'C' :  19
  'F' :  20
  'H' :  21
  'J' :  22
  'L' :  23
  'M' :  24
  'N' :  25
  'O' :  26
  'R' :  27
  'T' :  28
  'W' :  29
  '[' :  30
  ']' :  31
  '_' :  32
  'a' :  33
  'b' :  34
  'c' :  35
  'd' :  36
  'e' :  37
  'f' :  38
  'g' :  39
  'h' :  40
  'i' :  41
  'k' :  42
  'l' :  43
  'm' :  44
  'n' :  45
  'o' :  46
  'p' :  47
  'r' :  48
  's' :  49
  't' :  50
  'u' :  51
  'v' :  52
  'w' :  53
  'x' :  54
  'y' :  55
  '|' :  56
  'ű' :  57
  '̶' :  58
  '।' :  59
  '॥' :  60
  'ঁ' :  61
  'ং' :  62
  'ঃ' :  63
  'অ' :  64

Input text to Integer: 

'বজাও রে মোহন বাঁশি।\n' mapped to [ 97  82 107  73   1 101 113   1  99 115 106  94   1  97 107  61 103 108
  59   0]


## Create Training Data

In [28]:
seq_length= 150 #max number of characters that can be fed as a single input
examples_per_epoch = len(text)

#converts text (vector) into character index stream
#Reference: https://www.tensorflow.org/api_docs/python/tf/data/Dataset
char_dataset = tf.data.Dataset.from_tensor_slices(int_text)

In [29]:
#Create sequences from the individual characters. Our required size will be seq_length + 1 (character RNN)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [30]:
#Testing
print("Character Stream: \n")
for i in char_dataset.take(10):
  print(index2char[i.numpy()])  

print("\nSequence: \n")
for i in sequences.take(10):
  print(repr(''.join(index2char[i.numpy()])))  #use repr() for more clarity. str() keeps formatting it

Character Stream: 

ব
জ
া
ও
 
র
ে
 
ম
ো

Sequence: 

'বজাও রে মোহন বাঁশি।\nসারা দিবসক\nবিরহদহনদুখ,\nমরমক তিয়াষ নাশি।\nরিঝমনভেদন\nবাঁশরিবাদন\nকঁহা শিখলি রে কান?\nহানে থিরথির\nমরমঅবশকর\nলহু লহু মধুময় বাণ।\nধসধস করতহ\nউ'
'রহ বিয়াকুলু,\nঢুলু ঢুলু অবশনয়ান ;\nকত কত বরষক\nবাত সোঁয়ারয়,\nঅধীর করয় পরান।\nকত শত আশা\nপূরল না বঁধু,\nকত সুখ করল পয়ান।\nপহু গো কত শত\nপীরিতযাতন\nহিয়ে বিঁধাওল বা'
'ণ।\nহৃদয় উদাসয়,\nনয়ন উছাসয়\nদারুণ\nমধুময় গান।\nসাধ যায় বঁধূ,\nযমুনাবারিম\nডারিব\nদগধপরান।\nসাধ যায় পহু,\nরাখি চরণ তব\nহৃদয়মাঝ\nহৃদয়েশ,\nহৃদয়জুড়াওন\nবদনচন্দ্র তব\nহেরব'
'\nজীবনশেষ।\nসাধ যায়, ইহ\nচন্দ্রমকিরণে\nকুসুমিত\nকুঞ্জবিতানে\nবসন্তবায়ে\nপ্রাণ মিশায়ব\nবাঁশিক সুমধুর গানে।\nপ্রাণ ভৈবে মঝু\nবেণুগীতময়,\nরাধাময় তব\nবেণু।\nজয় জয় মাধব,'
'\nজয় জয় রাধা,\nচরণে\nপ্রণমে ভানু।\nশুনহ শুনহ বালিকা,\nরাখ কুসুমমালিকা,\nকুঞ্জ কুঞ্জ ফেরনু সখি শ্যামচন্দ্র নাহি রে।\nদুলই কুসুমমুঞ্জরী,\nভমর ফিরই গুঞ্জরি,\nঅলস য'
'মুনা বহয়ি যায় ললিত গীত গাহি রে।\nশশিসনাথ যামিনী,\nবিরহবিধুর কামিনী,\nকুসুমহার ভইল ভার— হৃদয় তার দাহিছে।\nঅধর উঠই কাঁপি


Target value: for each sequence of characters, we return that sequence, shifted one position to the right, along with the new character that is predicted to follow the sequence.

To create training examples of (input, target) pairs, we take the given sequence. The input is sequence with last word removed. Target is sequence with first word removed. Example: sequence: abc d ef input: abc d e target: bc d ef

In [31]:
def create_input_target_pair(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(create_input_target_pair)

In [32]:
#Testing
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(index2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(index2char[target_example.numpy()])))

Input data:  'বজাও রে মোহন বাঁশি।\nসারা দিবসক\nবিরহদহনদুখ,\nমরমক তিয়াষ নাশি।\nরিঝমনভেদন\nবাঁশরিবাদন\nকঁহা শিখলি রে কান?\nহানে থিরথির\nমরমঅবশকর\nলহু লহু মধুময় বাণ।\nধসধস করতহ\n'
Target data: 'জাও রে মোহন বাঁশি।\nসারা দিবসক\nবিরহদহনদুখ,\nমরমক তিয়াষ নাশি।\nরিঝমনভেদন\nবাঁশরিবাদন\nকঁহা শিখলি রে কান?\nহানে থিরথির\nমরমঅবশকর\nলহু লহু মধুময় বাণ।\nধসধস করতহ\nউ'


In [33]:
#Creating batches

BATCH_SIZE = 64

# Buffer used to shuffle the dataset 
# Reference: https://stackoverflow.com/questions/46444018/meaning-of-buffer-size-in-dataset-map-dataset-prefetch-and-dataset-shuffle
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>

## Building the Model

In [34]:
vocab_size = len(vocabulary)
embedding_dim = 256
rnn_units= 1024

3 Layers used:

Input Layer: Maps character to 256 dimension vector

GRU Layer: RNN of size 1024

Dense Layer: Output with same size as vocabulary

Since it is a character level RNN, we can use keras.Sequential model (All layers have single input and single output).

In [35]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, 
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

# Reference for theory: https://jhui.github.io/2017/03/15/RNN-LSTM-GRU/

In [36]:
lstm_model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [37]:
#Testing: shape
for input_example_batch, target_example_batch in dataset.take(1):
    example_prediction = lstm_model(input_example_batch)
    assert (example_prediction.shape == (BATCH_SIZE, seq_length, vocab_size)), "Shape error"
    #print(example_prediction.shape)

In [38]:
#model.summary() 
#check shapes if necessary

In [39]:
sampled_indices = tf.random.categorical(example_prediction[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

## Model Training

In [40]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#Loss Function reference: https://www.dlology.com/blog/how-to-use-keras-sparse_categorical_crossentropy/

example_loss  = loss(target_example_batch, example_prediction)
print("Prediction shape: ", example_prediction.shape)
print("Loss:      ", example_loss.numpy().mean())

Prediction shape:  (64, 150, 139)
Loss:       4.935026


In [41]:
lstm_model.compile(optimizer='adam', loss=loss)

In [42]:
lstm_dir_checkpoints= './training_checkpoints_LSTM'
checkpoint_prefix = os.path.join(lstm_dir_checkpoints, "checkpt_{epoch}") #name
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [43]:
EPOCHS=60 #increase number of epochs for better results (lesser loss)

In [44]:
history = lstm_model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
245/245 [==============================] - 21s 84ms/step - loss: 2.8583
Epoch 2/60
245/245 [==============================] - 21s 84ms/step - loss: 2.3891
Epoch 3/60
245/245 [==============================] - 21s 84ms/step - loss: 2.1671
Epoch 4/60
245/245 [==============================] - 21s 84ms/step - loss: 2.0206
Epoch 5/60
245/245 [==============================] - 20s 83ms/step - loss: 1.9280
Epoch 6/60
245/245 [==============================] - 20s 84ms/step - loss: 1.8658
Epoch 7/60
245/245 [==============================] - 21s 84ms/step - loss: 1.8206
Epoch 8/60
245/245 [==============================] - 20s 84ms/step - loss: 1.7857
Epoch 9/60
245/245 [==============================] - 20s 83ms/step - loss: 1.7564
Epoch 10/60
245/245 [==============================] - 20s 84ms/step - loss: 1.7307
Epoch 11/60
245/245 [==============================] - 21s 84ms/step - loss: 1.7082
Epoch 12/60
245/245 [==============================] - 20s 83ms/step - loss: 1.6870
E

In [45]:
tf.train.latest_checkpoint(lstm_dir_checkpoints)

'./training_checkpoints_LSTM/checkpt_60'

## Prediction

In [46]:
lstm_model = build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size=1)
lstm_model.load_weights(tf.train.latest_checkpoint(lstm_dir_checkpoints))
lstm_model.build(tf.TensorShape([1, None]))

lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            35584     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_2 (Dense)              (1, None, 139)            142475    
Total params: 5,425,035
Trainable params: 5,425,035
Non-trainable params: 0
_________________________________________________________________


In [47]:
def generate_text(model, start_string):
    num_generate = 1000 #Number of characters to be generated

    input_eval = [char2index[s] for s in start_string] #vectorising input
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [48]:
#Testing
#print(generate_text(lstm_model, start_string=u"ROMEO: "))

In [54]:
#Prediction with User Input
lstm_test = input("Enter your starting string: ")
print(generate_text(lstm_model, start_string=lstm_test))


Enter your starting string:  মৃত্যু স্পর্শ


মৃত্যু স্পর্শে তাঁর আত্ম-অনুভব।
মৃদু হেসে তরু ছায়া দিয়ে
দোঁহারের
পশ্চিমসীমায়,
সে বিরাট নাহিকো তাহার সত্য মূল্য তার নেই।
অন্ধকারে দেখা যায় আরো সবার মতো
সব গিয়াছে যে দুটি কথা —
এত যাওয়া ভালোবেসেছিনু মনে
ভানু মরণের স্বর্ণমাঝারে
নিঃশব্দ চরণে বরণ করিয়া প্রাণ।
পশ্চাতে যে প্রেম নিমেষে নিমেষে বুদ্ বুদের মধ্যে কেউ কোথাও নেই।
এমন সময় পাওয়া যেন
মনটাতে মোর বুকের কাছে বারি বারে,
সেখানে মিলেছে আঁধার ছায়ার তলে ;
সে হিসাব রাখাল বেয়ে ঘেঁষে ছিঁড়ে পিঠে,
সেখানে মাঠের পথের পথিক তুমি
চুপে চুপে,
সেই চলে গেল কত দিন একেবারে।
সব চেয়ে সভা বসে বনের পাশে
তবে কেন সে বাজে ভাবে,
কেন এ কেমন করে।
কোন্ খানে তার সেই
মেয়েটির হাসি,
অমন করে আছিস কোথা।
তোমার নীল আকাশের বাণী
নিরন্তর স্তরে স্তরে আঁধার চকিতে
নিম্নে সে দাঁড়ায়েছে দ্বারে গিয়া কী দিগন্তের মাঝে
দুর্গম বন্ধুর সমুখে প্রবাহিয়া উঠে মাতি।
সে যেন আমাদের প্রিয়ার আমারে
সে কথা নিশিদিন ধরি মোর বাজিয়ে দেব!
বিশ্বজনের পানে স্বপ্ন ভাঙো তটে ;
মনে হল কাজে লাগিবে বাতি
সে যেন কার তরে?
সবার সাথে আমাদের সত্য নহে,
সে কি অজানা ভাষা
আমার মনের কথা বলা হবে।
বাঁশির রব করি তাঁহার স্মৃতি,
মুগ্